In [2]:
from PIL import Image # used for loading images
import numpy as np
import os # used for navigating to image path
import imageio # used for writing images
import random
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from timeit import default_timer as timer
from tensorflow.keras import backend as K

import sys
sys.path.append("../python/")
from helpers import *

"""HP Tuning"""
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch

from tensorflow import keras

import IPython

from sklearn.preprocessing import LabelEncoder

import kerastuner as kt
from sklearn import ensemble
from sklearn import datasets
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection

In [3]:
"""Load numpy output files"""
images_size64_exp5_Pr_Po_Im = np.load('../../data/tidy/preprocessed_images/size64_exp5_Pr_Po_Im.npy', allow_pickle=True)
images_size64_exp5_Pr_Im = np.load('../../data/tidy/preprocessed_images/size64_exp5_Pr_Im.npy', allow_pickle=True)
images_size64_exp5_PrPo_Im = np.load('../../data/tidy/preprocessed_images/size64_exp5_PrPo_Im.npy', allow_pickle=True)
images_size64_exp5_Pr_PoIm = np.load('../../data/tidy/preprocessed_images/size64_exp5_Pr_PoIm.npy', allow_pickle=True)

images_size128_exp5_Pr_Po_Im = np.load('../../data/tidy/preprocessed_images/size128_exp5_Pr_Po_Im.npy', allow_pickle=True)
images_size128_exp5_Pr_Im = np.load('../../data/tidy/preprocessed_images/size128_exp5_Pr_Im.npy', allow_pickle=True)
images_size128_exp5_PrPo_Im = np.load('../../data/tidy/preprocessed_images/size128_exp5_PrPo_Im.npy', allow_pickle=True)
images_size128_exp5_Pr_PoIm = np.load('../../data/tidy/preprocessed_images/size128_exp5_Pr_PoIm.npy', allow_pickle=True)

In [4]:
NUM_CLASS = 2
NUM_CLASS_PR_PO_IM = 3
NUM_CHANNELS = 1
scenario_list = ["Pr_Im", "PrPo_Im", "Pr_PoIm", "Pr_Po_Im"]

In [5]:
image_set_64 = [images_size64_exp5_Pr_Im, images_size64_exp5_PrPo_Im, images_size64_exp5_Pr_PoIm, images_size64_exp5_Pr_Po_Im]
image_set_128 = [images_size128_exp5_Pr_Im, images_size128_exp5_PrPo_Im, images_size128_exp5_Pr_PoIm, images_size128_exp5_Pr_Po_Im]

In [6]:
input_image_shape_64 = getImageShape(images_size64_exp5_Pr_Im, num_channels = NUM_CHANNELS)
input_image_shape_128 = getImageShape(images_size128_exp5_Pr_Im, num_channels = NUM_CHANNELS)

In [7]:
def model_builder_64_2cl(hp):
  model = models.Sequential()
  
  ## Vary kernel size in first Conv Layer between 5 and 7
  hp_k_size = hp.Choice('kernel_size', values = [5, 7])
  
  # Tune the number of units in the first and second Dense layers
  # Choose an optimal value between 32-512
  hp_units_l1 = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  hp_units_l2 = hp.Int('units', min_value = 32, max_value = 512, step = 32)  
    
  dropout_rate_a = hp.Float('dropout', min_value = 0.0, max_value = 0.5, step = 0.1)
  dropout_rate_b = hp.Float('dropout', min_value = 0.0, max_value = 0.5, step = 0.1)
    
  # Experiment with "relu" and "tanh" activation f-ns
  hp_dl1_activation = hp.Choice('activation', values = ['relu', 'tanh'])
  hp_dl2_activation = hp.Choice('activation', values = ['relu', 'tanh'])
    
  model.add(layers.Conv2D(filters = 64, kernel_size = hp_k_size, strides = 2, activation="relu", padding="same", input_shape = input_image_shape_64))

  model.add(layers.MaxPooling2D(2))
  model.add(layers.Conv2D(128, 3, activation="relu", padding="same"))
  model.add(layers.Conv2D(128, 3, activation="relu", padding="same"))
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Conv2D(256, 3, activation="relu", padding="same"))
  model.add(layers.Conv2D(256, 3, activation="relu", padding="same"))
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Flatten())
  
  model.add(layers.Dense(units = hp_units_l1, activation = hp_dl1_activation))
  model.add(layers.BatchNormalization()) # Networks train faster & converge much more quickly
  model.add(layers.Dropout(dropout_rate_a))
  model.add(layers.Dense(units = hp_units_l2, activation = hp_dl2_activation))
  model.add(layers.Dropout(dropout_rate_b))
  model.add(keras.layers.Dense(NUM_CLASS, activation='softmax'))
    
  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.CategoricalCrossentropy(from_logits = True), #keras.losses.SparseCategoricalCrossentropy(from_logits = True)
                metrics = ['accuracy'])
  return model

def model_builder_64_3cl(hp):
  model = models.Sequential()
  
  ## Vary kernel size in first Conv Layer between 5 and 7
  hp_k_size = hp.Choice('kernel_size', values = [5, 7])
  
  # Tune the number of units in the first and second Dense layers
  # Choose an optimal value between 32-512
  hp_units_l1 = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  hp_units_l2 = hp.Int('units', min_value = 32, max_value = 512, step = 32)  
    
  dropout_rate_a = hp.Float('dropout', min_value = 0.0, max_value = 0.5, step = 0.1)
  dropout_rate_b = hp.Float('dropout', min_value = 0.0, max_value = 0.5, step = 0.1)
    
  # Experiment with "relu" and "tanh" activation f-ns
  hp_dl1_activation = hp.Choice('activation', values = ['relu', 'tanh'])
  hp_dl2_activation = hp.Choice('activation', values = ['relu', 'tanh'])
    
  model.add(layers.Conv2D(filters = 64, kernel_size = hp_k_size, strides = 2, activation="relu", padding="same", input_shape = input_image_shape_64))

  model.add(layers.MaxPooling2D(2))
  model.add(layers.Conv2D(128, 3, activation="relu", padding="same"))
  model.add(layers.Conv2D(128, 3, activation="relu", padding="same"))
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Conv2D(256, 3, activation="relu", padding="same"))
  model.add(layers.Conv2D(256, 3, activation="relu", padding="same"))
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Flatten())
  
  model.add(layers.Dense(units = hp_units_l1, activation = hp_dl1_activation))
  model.add(layers.BatchNormalization()) # Networks train faster & converge much more quickly
  model.add(layers.Dropout(dropout_rate_a))
  model.add(layers.Dense(units = hp_units_l2, activation = hp_dl2_activation))
  model.add(layers.Dropout(dropout_rate_b))
  model.add(keras.layers.Dense(NUM_CLASS_PR_PO_IM, activation='softmax'))
    
  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.CategoricalCrossentropy(from_logits = True), #keras.losses.SparseCategoricalCrossentropy(from_logits = True)
                metrics = ['accuracy'])
  return model

def model_builder_128_2cl(hp):
  model = models.Sequential()
  
  ## Vary kernel size in first Conv Layer between 5 and 7
  hp_k_size = hp.Choice('kernel_size', values = [5, 7])
  
  # Tune the number of units in the first and second Dense layers
  # Choose an optimal value between 32-512
  hp_units_l1 = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  hp_units_l2 = hp.Int('units', min_value = 32, max_value = 512, step = 32)  
    
  dropout_rate_a = hp.Float('dropout', min_value = 0.0, max_value = 0.5, step = 0.1)
  dropout_rate_b = hp.Float('dropout', min_value = 0.0, max_value = 0.5, step = 0.1)
    
  # Experiment with "relu" and "tanh" activation f-ns
  hp_dl1_activation = hp.Choice('activation', values = ['relu', 'tanh'])
  hp_dl2_activation = hp.Choice('activation', values = ['relu', 'tanh'])
    
  model.add(layers.Conv2D(filters = 64, kernel_size = hp_k_size, strides = 2, activation="relu", padding="same", input_shape = input_image_shape_128))

  model.add(layers.MaxPooling2D(2))
  model.add(layers.Conv2D(128, 3, activation="relu", padding="same"))
  model.add(layers.Conv2D(128, 3, activation="relu", padding="same"))
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Conv2D(256, 3, activation="relu", padding="same"))
  model.add(layers.Conv2D(256, 3, activation="relu", padding="same"))
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Flatten())
  
  model.add(layers.Dense(units = hp_units_l1, activation = hp_dl1_activation))
  model.add(layers.BatchNormalization()) # Networks train faster & converge much more quickly
  model.add(layers.Dropout(dropout_rate_a))
  model.add(layers.Dense(units = hp_units_l2, activation = hp_dl2_activation))
  model.add(layers.Dropout(dropout_rate_b))
  model.add(keras.layers.Dense(NUM_CLASS, activation='softmax'))
    
  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.CategoricalCrossentropy(from_logits = True), #keras.losses.SparseCategoricalCrossentropy(from_logits = True)
                metrics = ['accuracy'])
  return model

def model_builder_128_3cl(hp):
  model = models.Sequential()
  
  ## Vary kernel size in first Conv Layer between 5 and 7
  hp_k_size = hp.Choice('kernel_size', values = [5, 7])
  
  # Tune the number of units in the first and second Dense layers
  # Choose an optimal value between 32-512
  hp_units_l1 = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  hp_units_l2 = hp.Int('units', min_value = 32, max_value = 512, step = 32)  
    
  dropout_rate_a = hp.Float('dropout', min_value = 0.0, max_value = 0.5, step = 0.1)
  dropout_rate_b = hp.Float('dropout', min_value = 0.0, max_value = 0.5, step = 0.1)
    
  # Experiment with "relu" and "tanh" activation f-ns
  hp_dl1_activation = hp.Choice('activation', values = ['relu', 'tanh'])
  hp_dl2_activation = hp.Choice('activation', values = ['relu', 'tanh'])
    
  model.add(layers.Conv2D(filters = 64, kernel_size = hp_k_size, strides = 2, activation="relu", padding="same", input_shape = input_image_shape_128))

  model.add(layers.MaxPooling2D(2))
  model.add(layers.Conv2D(128, 3, activation="relu", padding="same"))
  model.add(layers.Conv2D(128, 3, activation="relu", padding="same"))
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Conv2D(256, 3, activation="relu", padding="same"))
  model.add(layers.Conv2D(256, 3, activation="relu", padding="same"))
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Flatten())
  
  model.add(layers.Dense(units = hp_units_l1, activation = hp_dl1_activation))
  model.add(layers.BatchNormalization()) # Networks train faster & converge much more quickly
  model.add(layers.Dropout(dropout_rate_a))
  model.add(layers.Dense(units = hp_units_l2, activation = hp_dl2_activation))
  model.add(layers.Dropout(dropout_rate_b))
  model.add(keras.layers.Dense(NUM_CLASS_PR_PO_IM, activation='softmax'))
    
  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.CategoricalCrossentropy(from_logits = True), #keras.losses.SparseCategoricalCrossentropy(from_logits = True)
                metrics = ['accuracy'])
  return model

In [8]:
tuner_64_2cl = kt.Hyperband(model_builder_64_2cl,
                     objective = 'loss', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'opt',
                     project_name = 'imageset_64px_2_class')

tuner_64_3cl = kt.Hyperband(model_builder_64_3cl,
                     objective = 'loss', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'opt',
                     project_name = 'imageset_64px_3_class')

tuner_128_2cl = kt.Hyperband(model_builder_128_2cl,
                     objective = 'loss', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'opt',
                     project_name = 'imageset_128px_2_class')

tuner_128_3cl = kt.Hyperband(model_builder_128_3cl,
                     objective = 'loss', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'opt',
                     project_name = 'imageset_128px_3_class')

In [9]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
#Best Hperparamter summary
#best_hps_summary = f"""
#The hyperparameter search for classification scenario {CLASSIFICATION_SCENARIO} is complete. The optimal number of units in the first densely-connected
#layer is {best_hps.get('units')}. The optimal learning rate for the optimizer
#is {best_hps.get('learning_rate')}. The optimal kernel size of the first convolution layer is {best_hps.get('kernel_size')}.
#The optimal dropout rate for the optimizer is {best_hps.get('dropout')}. The optimal activation layer for the optimizer is {best_hps.get('activation')}.
#"""
#
#best_hps_summary_path = model_path + '/best_hyperparameters_summary_scenario_' + CLASSIFICATION_SCENARIO + '.txt'
#print(best_hps_summary,  file=open(best_hps_summary_path, 'w'))

In [10]:
MODEL_PATH_64 = '../../results/models/64/'
MODEL_PATH_128 = '../../results/models/128/'
if not os.path.exists(MODEL_PATH_64): 
        os.makedirs(MODEL_PATH_64)
if not os.path.exists(MODEL_PATH_128): 
        os.makedirs(MODEL_PATH_128)        

In [11]:
def best_model_fit(tuner, model_path):
    tuner.search(training_images, training_labels, epochs = 10, validation_data = (validation_images, validation_labels), callbacks = [ClearTrainingOutput()])
    best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
    best_hps_dict = best_hps.values
    best_model = tuner.hypermodel.build(best_hps)
    best_model.fit(training_images, training_labels, epochs=10, validation_data=(validation_images, validation_labels))
    best_model_json = best_model.to_json()
    scenario_path = os.path.join(model_path, scenario_list[k])
    attr_path = os.path.join(scenario_path, 'attributes')
    best_hps_path = 'hyperparameters_'+scenario_list[k]+'.txt'
    model_summary_path = 'model_summary_'+scenario_list[k]+'.txt'
    model_to_json_path = 'model_to_json_'+scenario_list[k]+'.txt'
    if not os.path.exists(attr_path):
        os.makedirs(attr_path)
    with open(os.path.join(attr_path, best_hps_path), 'w') as f:
        f.write(json.dumps(best_hps_dict))
    with open(os.path.join(attr_path, model_summary_path), 'w') as f:
        best_model.summary(print_fn=lambda x: f.write(x + '\n'))
    with open(os.path.join(attr_path, model_to_json_path), 'w') as f:
        f.write(json.dumps(best_model_json))
    best_model.save(scenario_path)

In [12]:
k = 0
for is_64 in image_set_64:
    training_images_and_labels, test_images_and_labels = splitData(is_64, prop = 0.80, seed_num = 1)
    training_images, training_labels = getImageAndLabelArrays(training_images_and_labels)
    validation_images, validation_labels = getImageAndLabelArrays(test_images_and_labels)
    print("Beginning search for scenario: " + scenario_list[k])
    if(is_64 != images_size64_exp5_Pr_Po_Im):
        best_model_fit(tuner_64_2cl, MODEL_PATH_64)
    else:
        best_model_fit(tuner_64_3cl, MODEL_PATH_64)
    print("Search for scenario: " + scenario_list[k] + " is complete.")
    k=k+1

In [13]:
k = 0
for is_128 in image_set_128:
    training_images_and_labels, test_images_and_labels = splitData(is_128, prop = 0.80, seed_num = 1)
    training_images, training_labels = getImageAndLabelArrays(training_images_and_labels)
    validation_images, validation_labels = getImageAndLabelArrays(test_images_and_labels)
    print("Beginning search for scenario: " + scenario_list[k])
    if(is_128 != images_size128_exp5_Pr_Po_Im):
        best_model_fit(tuner_128_2cl, MODEL_PATH_128)
    else:
        best_model_fit(tuner_128_3cl, MODEL_PATH_128)
    print("Search for scenario: " + scenario_list[k] + " is complete.")
    k=k+1